# AutoPrognosis survival analysis

Welcome! This tutorial will walk you through the steps of selecting a model for a survival analysis task using AutoPrognosis.

### Setup

In [ ]:
# stdlib
import json
import warnings

import pandas as pd

# third party
from lifelines.datasets import load_rossi
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

### Import RiskEstimationStudy

RiskEstimationStudy is the engine that learns an ensemble of survival analysis pipelines and their hyperparameters automatically.

In [ ]:
# autoprognosis absolute
from autoprognosis.studies.risk_estimation import RiskEstimationStudy

### Load the target dataset

AutoPrognosis expects pandas.DataFrames as input.

For this example, we will use the [Rossi dataset](https://rdrr.io/cran/RcmdrPlugin.survival/man/Rossi.html).

In [ ]:
# third party
from lifelines.datasets import load_rossi

rossi = load_rossi()

X = rossi.drop(["week", "arrest"], axis=1)
Y = rossi["arrest"]
T = rossi["week"]

eval_time_horizons = [
    int(T[Y.iloc[:] == 1].quantile(0.25)),
    int(T[Y.iloc[:] == 1].quantile(0.50)),
    int(T[Y.iloc[:] == 1].quantile(0.75)),
]

### Create the risk estimation study

While AutoPrognosis provides default plugins, it allows the user to customize the plugins for the pipelines.

You can see the supported plugins below:

In [ ]:
# stdlib
# List the available plugins
import json
from pathlib import Path

# autoprognosis absolute
from autoprognosis.plugins import Plugins

print(json.dumps(Plugins().list_available(), indent=2))

We will set a few custom plugins for the pipelines and create the classifier study.

In [ ]:
workspace = Path("workspace")
workspace.mkdir(parents=True, exist_ok=True)

study_name = "test_risk_estimation_studies"

study = RiskEstimationStudy(
    study_name=study_name,
    dataset=rossi,
    target="arrest",
    time_to_event="week",
    time_horizons=eval_time_horizons,
    num_iter=10,  # DELETE THIS LINE FOR BETTER RESULTS.  number of BO iterations per estimator. Default: 50
    num_study_iter=1,  # DELETE THIS LINE FOR BETTER RESULTS.  number of outer optimization iterations. Default: 5
    risk_estimators=[
        "cox_ph",
        "lognormal_aft",
        "loglogistic_aft",
    ],  # DELETE THIS LINE FOR BETTER RESULTS.
    workspace=workspace,
    score_threshold=0.4,
)

### Search for the best ensemble


In [ ]:
study.run()

In [ ]:
# stdlib
import pprint

# autoprognosis absolute
from autoprognosis.utils.serialization import load_model_from_file
from autoprognosis.utils.tester import evaluate_survival_estimator

output = workspace / study_name / "model.p"

model = load_model_from_file(output)

metrics = evaluate_survival_estimator(model, X, T, Y, eval_time_horizons)

print(f"Model {model.name()}")
print(f"Score: ")

pprint.pprint(metrics)

## Serialization

In [ ]:
# autoprognosis absolute
from autoprognosis.utils.serialization import load_from_file, save_to_file

out = workspace / "tmp.bkp"

# Fit the model
model.fit(X, T, Y)

# Save
save_to_file(out, model)

# Reload
loaded_model = load_from_file(out)

print(loaded_model.name())

assert loaded_model.name() == model.name()

out.unlink()

## Congratulations!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards Machine learning and AI for medicine, you can do so in the following ways!

### Star AutoPrognosis on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the tools we're building.

- [Star AutoPrognosis](https://github.com/vanderschaarlab/autoprognosis)
- [Star HyperImpute](https://github.com/vanderschaarlab/hyperimpute)
